In [35]:
import pandas as pd
import numpy as np

In [37]:
korea = pd.read_csv('./nationseperate_Korea_190214.csv')
korea = korea.drop(columns='기간')
korea.head()

,국가,유효슈팅,반칙,점유율,날씨,기온,피파랭킹,팀연봉
0,한국,3,19,47,4,24,51,2306476
1,한국,7,7,53,3,20,51,2306476
2,한국,7,6,61,4,34,51,2306476
3,한국,6,14,67,3,19,51,2306476
4,한국,3,13,33,1,29,51,2306476


In [38]:
australia = pd.read_csv('./nationseperate_Australia_190214.csv')
australia = australia.drop(columns='기간')
australia.head()

,국가,유효슈팅,반칙,점유율,날씨,기온,피파랭킹,팀연봉
0,오스트레일리아,3,15,53,3,19,100,1534325
1,오스트레일리아,6,19,67,1,29,100,1534325
2,오스트레일리아,8,19,60,3,16,100,1534325
3,오스트레일리아,4,26,53,4,24,100,1534325
4,오스트레일리아,9,8,66,3,25,100,1534325


In [39]:
# 컬럼별로 숫자가 높을 수록 좋은건지 안 좋은 건지 바꿔줘야함(통일시켜야함)
# 반칙 수, 랭킹 => 높을수록 안좋음...
# 랭킹은 211위가 꼴찌라 212에서 빼면 역순.
korea['피파랭킹'] = 212-korea['피파랭킹']
australia['피파랭킹'] = 212-australia['피파랭킹']


In [40]:
# 위에 값이 이상해서 그냥 -를 붙여보자
korea['반칙'] = -korea['반칙']
australia['반칙'] = -australia['반칙']

In [41]:
# pd.set_option('display.float_format', '{:.5f}'.format) 지수없애기

In [44]:
# 날씨, 국가는 카테고리로 일단 바꾸기
korea['날씨'] = korea['날씨'].astype('category')
korea['국가'] = korea['국가'].astype('category')

australia['날씨'] = australia['날씨'].astype('category')
australia['국가'] = australia['국가'].astype('category')

In [45]:
korea.head()

,국가,유효슈팅,반칙,점유율,날씨,기온,피파랭킹,팀연봉
0,한국,3,-19,47,4,24,161,2306476
1,한국,7,-7,53,3,20,161,2306476
2,한국,7,-6,61,4,34,161,2306476
3,한국,6,-14,67,3,19,161,2306476
4,한국,3,-13,33,1,29,161,2306476


In [46]:
australia.head()

,국가,유효슈팅,반칙,점유율,날씨,기온,피파랭킹,팀연봉
0,오스트레일리아,3,-15,53,3,19,112,1534325
1,오스트레일리아,6,-19,67,1,29,112,1534325
2,오스트레일리아,8,-19,60,3,16,112,1534325
3,오스트레일리아,4,-26,53,4,24,112,1534325
4,오스트레일리아,9,-8,66,3,25,112,1534325


In [47]:
# 테스트 : 연봉 값을 표준화해서 해보자
australia['팀연봉'] = np.log1p(australia['팀연봉'])
australia['예측값'] = australia['유효슈팅'] + australia['반칙'] + australia['점유율'] + australia['기온'] + australia['피파랭킹'] + np.log1p(australia['팀연봉'])

korea['팀연봉'] = np.log1p(korea['팀연봉'])
korea['예측값'] = korea['유효슈팅'] + korea['반칙'] + korea['점유율'] + korea['기온'] + korea['피파랭킹'] + np.log1p(korea['팀연봉'])

In [59]:
korea = korea.set_index('국가')
korea.head()

,유효슈팅,반칙,점유율,날씨,기온,피파랭킹,팀연봉,예측값
국가,,,,,,,,
한국,3,-19,47,4,24,161,14.651232,218.75055
한국,7,-7,53,3,20,161,14.651232,236.75055
한국,7,-6,61,4,34,161,14.651232,259.75055
한국,6,-14,67,3,19,161,14.651232,241.75055
한국,3,-13,33,1,29,161,14.651232,215.75055


In [60]:
australia = australia.set_index('국가')
australia.head()

,유효슈팅,반칙,점유율,날씨,기온,피파랭킹,팀연봉,예측값
국가,,,,,,,,
오스트레일리아,3,-15,53,3,19,112,14.243602,174.72416
오스트레일리아,6,-19,67,1,29,112,14.243602,197.72416
오스트레일리아,8,-19,60,3,16,112,14.243602,179.72416
오스트레일리아,4,-26,53,4,24,112,14.243602,169.72416
오스트레일리아,9,-8,66,3,25,112,14.243602,206.72416


In [50]:
from sklearn.model_selection import train_test_split

In [79]:
# 한국 통으로
X_train = korea[['유효슈팅','반칙','점유율','날씨','기온','피파랭킹','팀연봉']]
Y_train = korea['예측값']

In [80]:
print(X_train.shape) 
print(Y_train.shape) 

(12, 7)
(12,)


In [81]:
# 호주 통으로
X_test = australia[['유효슈팅','반칙','점유율','날씨','기온','피파랭킹','팀연봉']]
Y_test = australia['예측값']

In [82]:
print(X_test.shape) 
print(Y_test.shape) 

(12, 7)
(12,)


In [83]:
# RMSLE
from sklearn.metrics import make_scorer
def rmsle(predicted_values, actual_values):
    # Numpy로 배열 형태로 변경
    predicted_values = np.array(predicted_values)
    actual_values = np.array(actual_values)
    
    # 에측값과 실제값에 1을 더하고 log를 씌운다.
    log_predict = np.log(predicted_values + 1)
    log_actual = np.log(actual_values + 1)
    
    # 위에서 계싼된 예측값에서 실제값을 빼주고 제곱을 해준다.
    difference = log_predict - log_actual
    difference = np.square(difference)
    
    # 평균을 구한다
    mean_difference = difference.mean()
    
    # 루트를 씌운다
    score = np.sqrt(mean_difference)
    
    return score

rmsle_scorer = make_scorer(rmsle)
rmsle_scorer

make_scorer(rmsle)

In [84]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

k_fold = KFold(n_splits = 10, shuffle = True, random_state = 0) # 고정된 상태로 10번 섞어서 테스트해라

In [85]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [86]:
model = GradientBoostingRegressor(n_estimators=1000,alpha=0.1, random_state=0) # 초기값은 1000, alpha(10%씩줄이며 시도)
# model = RandomForestRegressor(n_estimators=100, n_jobs=-1,random_state=0)
model

GradientBoostingRegressor(alpha=0.1, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=1000, n_iter_no_change=None, presort='auto',
             random_state=0, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [87]:
model2 = GradientBoostingRegressor(n_estimators=1000,alpha=0.1, random_state=0) # 초기값은 1000, alpha(10%씩줄이며 시도)
# model = RandomForestRegressor(n_estimators=100, n_jobs=-1,random_state=0)
model2

GradientBoostingRegressor(alpha=0.1, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=1000, n_iter_no_change=None, presort='auto',
             random_state=0, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [88]:
# Training
model.fit(X_train, Y_train)

GradientBoostingRegressor(alpha=0.1, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=1000, n_iter_no_change=None, presort='auto',
             random_state=0, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [89]:
# Training
model2.fit(X_test, Y_test)

GradientBoostingRegressor(alpha=0.1, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=1000, n_iter_no_change=None, presort='auto',
             random_state=0, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [73]:
# %time score = cross_val_score(model, X_train, Y_train, cv = k_fold, scoring = rmsle_scorer)
# score = score.mean()
# print("Score = ", score)

In [110]:
# 예측
pre = model.predict([[3,-19,47,1,23,50,13]])
print(pre)

[221.91378705]


In [111]:
# 예측
pre2 = model2.predict([[4,-26,53,1,23,50,13]])
print(pre2)

[171.43139091]


In [96]:
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# import seaborn as sns
# %matplotlib inline

# import warnings
# warnings.filterwarnings('ignore')

In [109]:
# # 예측한 데이터를 시각화
# fig, (ax1, ax2) = plt.subplots(ncols=2)
# fig.set_size_inches(12, 5)
# sns.distplot(Y_train, ax = ax1, bins=50) # 50개정도로 나눔
# ax1.set(title = 'Train')
# sns.distplot(pre, ax = ax2, bins = 50)
# ax2.set(title = 'Test')